# Import

In [2]:
import os
import re
import sys
from collections import defaultdict

import pandas as pd

# Declare Path

In [3]:
root_dir = '/Users/jake/OneDrive - leverage innovative users/Documents/News_Item/University/'
dirs = [x for x in os.listdir(root_dir) if x != '.DS_Store']
files = {}

d = 'a1_submission'
dir_path = os.path.join(root_dir, d)
files[d] = [os.path.join(root_dir, d, x) for x in os.listdir(dir_path) if x != '.DS_Store' and x.endswith('.tsv')]

files

{'a1_submission': ['/Users/jake/OneDrive - leverage innovative users/Documents/News_Item/University/a1_submission/2017_submission.tsv',
  '/Users/jake/OneDrive - leverage innovative users/Documents/News_Item/University/a1_submission/2018_submission.tsv',
  '/Users/jake/OneDrive - leverage innovative users/Documents/News_Item/University/a1_submission/2016_submission.tsv']}

# Load Files

In [4]:
pd.set_option('display.max_row', 50)
df = defaultdict(dict)
for dir_name, file_list in files.items():
    key = dir_name.split('_')[-1]
    for file in file_list:
        file_name = re.search(r'(\d{4})', file.split('/')[-1]).group(1)
        df[file_name] = pd.read_csv(file, sep='\t')

df

defaultdict(dict,
            {'2017':       기준년도       학교종류 설립구분  지역  상태             학교 정원구분       전형유형  \
             0     2017        대학교   사립  경남  기존  가야대학교(김해) _본교  정원내  학생부위주(교과)   
             1     2017        대학교   사립  경남  기존  가야대학교(김해) _본교  정원내  학생부위주(교과)   
             2     2017        대학교   사립  경남  기존  가야대학교(김해) _본교  정원내       수능위주   
             3     2017        대학교   사립  경남  기존  가야대학교(김해) _본교  정원외       수능위주   
             4     2017        대학교   사립  경남  기존  가야대학교(김해) _본교  정원외         기타   
             5     2017        대학교   사립  경남  기존  가야대학교(김해) _본교  정원외  학생부위주(교과)   
             6     2017        대학교   사립  경남  기존  가야대학교(김해) _본교  정원외  학생부위주(교과)   
             7     2017        대학교   사립  경남  기존  가야대학교(김해) _본교  정원외  학생부위주(교과)   
             8     2017        대학교   사립  경남  기존  가야대학교(김해) _본교  정원외         기타   
             9     2017        대학교   사립  경남  기존  가야대학교(김해) _본교  정원외       수능위주   
             10    2017        대학교   사립  경기  기존      가천대학교 _본교  정원내  학생부

# Functions

In [5]:
def convert_str2int(d, cols):
    d[cols] = d[cols].applymap(lambda x: pd.to_numeric(x.replace(',', '') if type(x)!=int else x))

# Index and Columns

In [6]:
# 13 univ
univ_names = ['서울', '연세', '고려', '서강', '성균관', '한양', '중앙', '경희', '한국외국어', '서울시립', '건국', '동국', '홍익']
univ = [u + '대학교 _본교' for u in univ_names]

idx_2016 = df['2016']['학교'].isin(univ)
idx_2017 = df['2017']['학교'].isin(univ)
idx_2018 = df['2018']['학교'].isin(univ)

cols_sub = ['수시 모집인원', '수시 등록인원', '정시 모집인원',
       '정시 등록인원', '추가 모집인원', '추가 등록인원', '최종 모집인원',
       '최종 등록인원',]

# '수시 등록률', '정시 등록률', '추가 등록률', 

agg_sub = {c:'sum' for c in cols_sub}

In [7]:
df['2018'].columns

Index(['기준년도', '학교종류', '설립구분', '지역', '상태', '학교', '정원구분', '전형유형', '전형명 (대분류)',
       '전형명 (중분류)', '전형명 (소분류)', '수시 모집인원', '수시 등록인원', '수시 등록률', '정시 모집인원',
       '정시 등록인원', '정시 등록률', '추가 모집인원', '추가 등록인원', '추가 등록률', '최종 모집인원',
       '최종 등록인원', '최종 등록률'],
      dtype='object')

# Pre-process

In [8]:
# df['2016_submission'][cols_sub] = df['2016_submission'][cols_sub].applymap(lambda x: pd.to_numeric(x.replace(',', '') if type(x)!=int else x))
convert_str2int(df['2016'], cols_sub)
convert_str2int(df['2017'], cols_sub)
convert_str2int(df['2018'], cols_sub)

# Transform

## Grouping

In [9]:
grp_2016 = df['2016'].loc[idx_2016].groupby(['학교', '전형유형', '전형명 (소분류)']).agg({'최종 등록인원':'sum'})
grp_2017 = df['2017'].loc[idx_2017].groupby(['학교', '전형유형', '전형명 (소분류)']).agg({'최종 등록인원':'sum'})
grp_2018 = df['2018'].loc[idx_2018].groupby(['학교', '전형유형', '전형명 (소분류)']).agg({'최종 등록인원':'sum'})

tmp = grp_2016.merge(grp_2017, left_index=True, right_index=True, suffixes=('_2016','_2017'), how='outer').fillna(0)
grp_2018 = grp_2018.rename(columns={'최종 등록인원':'최종 등록인원_2018'})
grp_sub = tmp.merge(grp_2018, left_index=True, right_index=True, how='outer').fillna(0)

del grp_2016
del grp_2017
del grp_2018
del tmp


# grp_sub['변화율_2017'] = round((grp_sub['최종 등록인원_2017'] - grp_sub['최종 등록인원_2016']) / grp_sub['최종 등록인원_2017'] * 100, 2)
pd.set_option('display.max_rows', None)
grp_sub

최종 등록인원_2016  \
학교           전형유형      전형명 (소분류)                                      
건국대학교 _본교    기타        12년 과정 이수자(3월)                           6.0   
                       부모 모두 외국인인 외국인(3월)                     239.0   
                       북한이탈주민                                   3.0   
                       재외국민과 외국인(2% 이내)                        48.0   
             논술위주      KU논술우수자전형                              480.0   
             수능위주      KU고른기회전형-기초생활수급자및차상위급여대상자 정시             0.0   
                       KU고른기회전형-농어촌 정시                          0.0   
                       KU고른기회전형-농어촌학생 정시                        6.0   
                       KU고른기회전형-특성화고교출신자 정시                     1.0   
                       KU일반전형-가군                              524.0   
                       KU일반전형-나군                              729.0   
                       KU일반전형-나군 체육교육 수정                        0.0   
                       KU일반전형-다군                              114.0   
                       KU일반전형-다군                                0.0   
                       KU일반학생전형(추가모집)                           0.0   
                       고른기회전형Ⅰ(기초생활수급자및차상위급여수급자)_정시이월           0.0   
                       고른기회전형Ⅰ(농어촌학생)_정시이월                      0.0   
                       고른기회전형Ⅰ(특성화고교졸업자)_정시이월                   0.0   
             실기위주      KU예체능우수자전형[유형1-연기]                      15.0   
                       KU예체능우수자전형[유형2-체육]                      15.0   
                       KU예체능우수자전형[유형3-디자인]                      0.0   
                       KU일반전형-가군 영화·애니메이션학과-애니메이션               0.0   
                       KU일반전형-가군 영화·애니메이션학과-연출·제작               0.0   
                       KU일반전형-가군 예술디자인대(1-4-5)                  0.0   
                       KU일반전형-나군 예술디자인대(1-4-5)                  0.0   
                       KU일반전형-나군 예술디자인대학 1:4:5 그룹             150.0   
                       KU일반전형-나군 체육교육                          27.0   
                       KU일반전형-나군 체육교육과                          0.0   
                       KU일반전형-나군 현대미술                          35.0   
                       KU일반전형-나군(예체능계) 1-4-5 그룹                 0.0   
                       KU일반전형-다군 1-4-5 그룹 연출제작                  0.0   
                       KU일반전형-다군 애니메이션 1-4-5                    0.0   
                       KU일반전형-다군 영화·애니메이션학과                    30.0   
                       KU일반전형-다군 음악교육                          27.0   
                       KU일반전형-다군 음악교육 1.5-4-5                   0.0   
             학생부위주(교과) KU고른기회전형[유형6-특수교육대상자]                   17.0   
                       KU교과우수자전형                              119.0   
             학생부위주(종합) KU고른기회전형-다군 특성화고졸재직자 정시                  0.0   
                       KU고른기회전형-특성화졸재직자 정시                      7.0   
                       KU고른기회전형[유형1-사회적배려대상자]                  44.0   
                       KU고른기회전형[유형2-기초생활수급자및차상위급여대상자]          56.0   
                       KU고른기회전형[유형3-농어촌학생]                     54.0   
                       KU고른기회전형[유형4-특성화고교출신자]                  41.0   
                       KU고른기회전형[유형5-특성화고졸재직자]                  61.0   
                       KU고른기회전형[유형6-특수교육대상자]                    0.0   
                       KU고른기회전형[유형7-지역인재]                     115.0   
                       KU자기추천전형                               573.0   
                       KU학교추천전형                                 0.0   
                       고른기회전형Ⅰ (국가보훈대상자)                        0.0   
                       고른기회전형Ⅰ (기초생활수급자및차상위급여수급자)               0.0   
                       고른기회전형Ⅰ (농어촌학생)                          0.0   
                       고른기회전형Ⅰ (특성화고교등을졸업한재직자)                  0.0   
                       고른기회전형Ⅰ (특성화고교졸업자)                       0.0   
                       고른기회전형Ⅰ (특수교육대상자)                        0.0   
                       고른기회전형Ⅰ(특성화고교등을졸업한재직자)_정시이월              0.0   
        

## 수시정시 3년 추이

In [10]:
cols_type = ['수시 모집인원', '수시 등록인원', '정시 모집인원',
       '정시 등록인원', '추가 모집인원', '추가 등록인원', '최종 모집인원',
       '최종 등록인원',]
agg_type = {c:'sum' for c in cols_type}

school = '성균관대학교 _본교'

idx_school_2016 = df['2016']['학교']==school
idx_school_2017 = df['2017']['학교']==school
idx_school_2018 = df['2018']['학교']==school

row0 = df['2016'].loc[idx_2016 & idx_school_2016].agg(agg_type)
row1 = df['2017'].loc[idx_2017 & idx_school_2017].agg(agg_type)
row2 = df['2018'].loc[idx_2018 & idx_school_2018].agg(agg_type)

df_susi_jeongsi = pd.DataFrame(columns=cols_type, data=[row0, row1, row2], index=[2016, 2017, 2018]); df_susi_jeongsi

,수시 모집인원,수시 등록인원,정시 모집인원,정시 등록인원,추가 모집인원,추가 등록인원,최종 모집인원,최종 등록인원
2016,3249,3069,1044,1058,0,0,4128,4100
2017,3243,3162,922,939,0,0,4086,4077
2018,3449,3241,902,913,0,0,4158,4154


In [11]:
# 서울대: 72.279, 71.160, 74.466 / 28.053, 29.078, 25.533
# 연세대: 66.496425	34.167518 / 65.235279	34.764721 / 67.139241	33.265823
# 고려대: 73.147341	27.179599 / 74.175347	26.280382 / 81.989785	18.010215
# 서강대: 69.791109	31.922871 / 70.582066	30.466702 / 74.619822	25.852124
# 성균관대: 74.853659	25.804878 / 77.557027	23.031641 / 78.021184	21.978816

df_susi_jeongsi['수시 비율'] = df_susi_jeongsi['수시 등록인원'] / df_susi_jeongsi['최종 등록인원'] * 100
df_susi_jeongsi['정시 비율'] = df_susi_jeongsi['정시 등록인원'] / df_susi_jeongsi['최종 등록인원'] * 100
df_susi_jeongsi.index.name = '연도'
df_susi_jeongsi

,수시 모집인원,수시 등록인원,정시 모집인원,정시 등록인원,추가 모집인원,추가 등록인원,최종 모집인원,최종 등록인원,수시 비율,정시 비율
연도,,,,,,,,,,
2016,3249,3069,1044,1058,0,0,4128,4100,74.853659,25.804878
2017,3243,3162,922,939,0,0,4086,4077,77.557027,23.031641
2018,3449,3241,902,913,0,0,4158,4154,78.021184,21.978816


## 정원초과 유일: 카이스트

In [ ]:
# cols_type = ['수시 모집인원', '수시 등록인원', '정시 모집인원',
       '정시 등록인원', '추가 모집인원', '추가 등록인원', '최종 모집인원',
       '최종 등록인원',]
agg_type = {c:'sum' for c in cols_type}
grp_not_seoul = df['2018'].loc[~idx_2018].groupby('학교').agg(agg_type)
idx_kaist = grp_not_seoul['최종 모집인원'] < grp_not_seoul['최종 등록인원']
grp_not_seoul.loc[idx_kaist]

In [13]:
grp_seoul = df['2018'].loc[idx_2018].groupby('학교').agg(agg_type)
grp_seoul.loc[grp_seoul['수시 모집인원'] > grp_seoul['수시 등록인원']]

,수시 모집인원,수시 등록인원,정시 모집인원,정시 등록인원,추가 모집인원,추가 등록인원,최종 모집인원,최종 등록인원
학교,,,,,,,,
건국대학교 _본교,2226,2075,1437,1397,28,17,3530,3480
경희대학교 _본교,4383,4215,1498,1495,0,0,5781,5688
고려대학교 _본교,3955,3692,802,811,0,0,4571,4503
동국대학교 _본교,2495,2240,938,913,22,9,3340,3162
서강대학교 _본교,1518,1423,501,493,0,0,1922,1907
서울대학교 _본교,2738,2546,886,873,0,0,3443,3419
서울시립대학교 _본교,1137,1036,846,840,0,0,1890,1869
성균관대학교 _본교,3449,3241,902,913,0,0,4158,4154
연세대학교 _본교,2957,2652,1313,1314,0,0,3966,3950


In [14]:
grp_seoul_kaist = grp_not_seoul.loc[idx_kaist].append(grp_seoul); grp_seoul_kaist
grp_seoul_kaist['최종 등록률'] = grp_seoul_kaist['최종 등록인원'] / grp_seoul_kaist['최종 모집인원'] * 100; grp_seoul_kaist
grp_seoul_kaist = grp_seoul_kaist.sort_values('최종 등록률', ascending=False); grp_seoul_kaist

,수시 모집인원,수시 등록인원,정시 모집인원,정시 등록인원,추가 모집인원,추가 등록인원,최종 모집인원,최종 등록인원,최종 등록률
학교,,,,,,,,,
한국과학기술원 _본교,690,723,20,25,0,0,710,748,105.352113
성균관대학교 _본교,3449,3241,902,913,0,0,4158,4154,99.903800
연세대학교 _본교,2957,2652,1313,1314,0,0,3966,3950,99.596571
중앙대학교 _본교,3412,3357,1096,1104,0,0,4127,4103,99.418464
한양대학교 _본교,2603,2565,984,995,0,0,3555,3533,99.381153
서울대학교 _본교,2738,2546,886,873,0,0,3443,3419,99.302933
한국외국어대학교 _본교,2311,2263,1761,1744,0,0,4016,3986,99.252988
서강대학교 _본교,1518,1423,501,493,0,0,1922,1907,99.219563
홍익대학교 _본교,1835,1791,1055,1032,0,0,2854,2823,98.913805


# Export TSV

In [15]:
# grp_export = grp_sub.reset_index()
# file_name = 'submission_all.tsv'
# file1 = os.path.join(root_dir, file_name)
# grp_export.to_csv(file1, sep='\t', index=False)

# name2 = 'a1-3_susi_jeongsi.tsv'
# file2 = os.path.join(root_dir, name2)
# df_susi_jeongsi.to_csv(file2, sep='\t', index=True)

# name3 = 'a1-6_registration_rate.tsv'
# file3 = os.path.join(root_dir, name3)
# grp_seoul_kaist.to_csv(file3, sep='\t', index=True)

# -------------------------------------------------------------------------------------------

In [30]:
tmp = df['2018'].loc[idx_2018].groupby(['학교', '전형유형', '전형명 (대분류)', '전형명 (중분류)',  '전형명 (소분류)']).agg({'최종 등록인원': 'sum'}); tmp

최종 등록인원
학교           전형유형      전형명 (대분류)            전형명 (중분류)           전형명 (소분류)                              
건국대학교 _본교    기타        재외국민 외국인(정원외)        12년 과정 이수자(3월)      12년 과정 이수자(3월)                        6
                                            부모모두외국인인 외국인(3월)    부모 모두 외국인인 외국인(3월)                  134
                                            북한이탈주민              북한이탈주민                                1
                                            재외국민과 외국인(2% 이내)    재외국민과 외국인(2% 이내)                     44
             논술위주      일반전형                 일반학생                KU논술우수자전형                           463
             수능위주      기초생활수급자 및 차상위계층(정원외) 저소득층(기회균형선발)        고른기회전형Ⅰ(기초생활수급자및차상위급여수급자)_정시이월        3
                       농어촌 학생(정원외)          농어촌 학생              고른기회전형Ⅰ(농어촌학생)_정시이월                   2
                       일반전형                 일반학생                KU일반전형-가군                           456
                                                                KU일반전형-나군                           534
                                                                KU일반전형-다군                           143
                                                                KU일반학생전형(추가모집)                       16
                       특성화고교 졸업자(정원외)       특성화고교 졸업자           고른기회전형Ⅰ(특성화고교졸업자)_정시이월                7
             실기위주      일반전형                 일반학생                KU일반전형-가군 영화·애니메이션학과-애니메이션           30
                                                                KU일반전형-가군 영화·애니메이션학과-연출·제작           15
                                                                KU일반전형-가군 예술디자인대(1-4-5)             105
                                                                KU일반전형-나군 예술디자인대(1-4-5)              42
                                                                KU일반전형-나군 체육교육과                      26
                                                                KU일반전형-다군 음악교육                       20
                       특기자                  기타                  KU예체능우수자전형[유형1-연기]                   15
                                            미술                  KU예체능우수자전형[유형3-디자인]                  20
                                            체육                  KU예체능우수자전형[유형2-체육]                   12
             학생부위주(종합) 고른기회 대상자(정원내)        국가보훈대상자             고른기회전형Ⅰ (국가보훈대상자)                    17
                                            특성화고 등을 졸업한 재직자     고른기회전형Ⅰ (특성화고교등을졸업한재직자)               6
                       기초생활수급자 및 차상위계층(정원외) 저소득층(기회균형선발)        고른기회전형Ⅰ (기초생활수급자및차상위급여수급자)           57
                       농어촌 학생(정원외)          농어촌 학생              고른기회전형Ⅰ (농어촌학생)                      76
                       대학별독자적기준             기타                  KU자기추천전형                            679
                                                                KU학교추천전형                            366
                                                                고른기회전형Ⅱ                              35
                       장애인등대상자(정원외)         장애인등대상자             고른기회전형Ⅰ (특수교육대상자)                    17
                       특성화고 등을 졸업한 재직자(정원외) 특성화고 등을 졸업한 재직자     고른기회전형Ⅰ (특성화고교등을졸업한재직자)             102
                                                                고른기회전형Ⅰ(특성화고교등을졸업한재직자)_정시이월          11
                       특성화고교 졸업자(정원외)       특성화고교 졸업자           고른기회전형Ⅰ (특성화고교졸업자)                   20
경희대학교 _본교    기타        재외국민 외국인(정원외)        12년 과정 이수자(3월)      12년 과정 이수자(3월)                       59
                                            부모모두외국인인 외국인(3월)    부모 모두 외국인인 외국인(3월)                  411
                                            북한이탈주민              북한이탈주민                                5
                                            재외국민과 외국인(2% 이내)    재외국민과 외국인(2% 이내)                     92
             논술위주      일반전형                 일반학생                논술우수자전형                             819
             수능위주      일반전형             

In [31]:
tmp = tmp.reset_index(); tmp
tmp.groupby(['학교']).agg({'전형명 (소분류)': 'count'})

,전형명 (소분류)
학교,
건국대학교 _본교,32
경희대학교 _본교,31
고려대학교 _본교,19
동국대학교 _본교,32
서강대학교 _본교,15
서울대학교 _본교,11
서울시립대학교 _본교,14
성균관대학교 _본교,20
연세대학교 _본교,23


In [37]:
tmp2 = tmp.loc[tmp['전형명 (대분류)'].str.contains('외국인')].groupby(['학교', '전형명 (대분류)']).agg({'최종 등록인원':'sum'}); tmp2

,,최종 등록인원
학교,전형명 (대분류),
건국대학교 _본교,재외국민 외국인(정원외),185
경희대학교 _본교,재외국민 외국인(정원외),567
고려대학교 _본교,재외국민 외국인(정원외),473
동국대학교 _본교,재외국민 외국인(정원외),201
서강대학교 _본교,재외국민 외국인(정원외),243
서울대학교 _본교,재외국민 외국인(정원외),80
서울시립대학교 _본교,재외국민 외국인(정원외),31
성균관대학교 _본교,재외국민 외국인(정원외),613
연세대학교 _본교,재외국민 외국인(정원외),343


In [38]:
tmp2.sort_values('최종 등록인원', ascending=False)

,,최종 등록인원
학교,전형명 (대분류),
성균관대학교 _본교,재외국민 외국인(정원외),613
경희대학교 _본교,재외국민 외국인(정원외),567
고려대학교 _본교,재외국민 외국인(정원외),473
한국외국어대학교 _본교,재외국민 외국인(정원외),422
한양대학교 _본교,재외국민 외국인(정원외),416
중앙대학교 _본교,재외국민 외국인(정원외),364
연세대학교 _본교,재외국민 외국인(정원외),343
서강대학교 _본교,재외국민 외국인(정원외),243
동국대학교 _본교,재외국민 외국인(정원외),201


In [17]:
pd.set_option('display.max_row', None)
df['2016'].loc[idx_2016].groupby(['학교', '전형유형']).agg(agg_sub)

수시 모집인원  수시 등록인원  정시 모집인원  정시 등록인원  추가 모집인원  추가 등록인원  \
학교           전형유형                                                              
건국대학교 _본교    기타             308      296        0        0        0        0   
             논술위주           484      484        0        0        0        0   
             수능위주             0        0     1377     1384        0        0   
             실기위주            30       30      269      269        0        0   
             학생부위주(교과)      206      141        0        0        0        0   
             학생부위주(종합)     1027      951       12        7        0        0   
경희대학교 _본교    기타             715      585        0        0        0        0   
             논술위주           925      920        0        0        0        0   
             수능위주             0        0     1649     1645        0        0   
             실기위주           355      344      355      355        0        0   
             학생부위주(종합)     1880     1695      285      282        0        0   
고려대학교 _본교    기타             577      574        0        0        0        0   
             논술위주          1110     1110        0        0        0        0   
             수능위주             0        0     1235     1247        0        0   
             실기위주           615      584        0        0        0        0   
             학생부위주(교과)      630      584        0        0        0        0   
             학생부위주(종합)      634      504        0        0        0        0   
동국대학교 _본교    기타             322      229        0        0        0        0   
             논술위주           499      499        0        0        0        0   
             수능위주             0        0     1377     1373        0        0   
             실기위주           118      118        0        0        0        0   
             학생부위주(교과)      396      345        4        3        0        0   
             학생부위주(종합)      615      581        0        0        0        0   
서강대학교 _본교    기타             189      189        0        0        0        0   
             논술위주           385      384        0        0        0        0   
             수능위주             0        0      592      596        0        0   
             실기위주           138      123        0        0        0        0   
             학생부위주(종합)      620      607        0        0        0        0   
서울대학교 _본교    기타              62       62        1        1        0        0   
             수능위주             0        0      917      921        0        0   
             학생부위주(종합)     2490     2329       18        6        0        0   
서울시립대학교 _본교  기타               0        0       11       11        0        0   
             논술위주           190      190        0        0        0        0   
             수능위주             0        0     1078     1053        0        0   
             실기위주             0        0      115      110        0        0   
             학생부위주(종합)      557      498        0        0        0        0   
성균관대학교 _본교   기타             517      517        0        0        0        0   
             논술위주          1311     1224        0        0        0        0   
             수능위주             0        0     1044     1058        0        0   
             실기위주            97       97        0        0        0        0   
             학생부위주(종합)     1324     1231        0        0        0        0   
연세대학교 _본교    기타             352      352        0        0        0        0   
             논술위주           683      681        0        0        0        0   
             수능위주             0        0     1239     1235        0        0   
             실기위주           970      875      103      103        0        0   
             학생부위주(교과)      257      152        0        0        0        0   
             학생부위주(종합)      681      544        0        0        0        0   
중앙대학교 _본교    기타             333      332        0        0        0        0   
             논술위주       

In [18]:
pd.set_option('display.max_column', None)
df['2016'].loc[idx_2016]

,기준년도,학교종류,설립구분,지역,상태,학교,정원구분,전형유형,전형명 (대분류),전형명 (중분류),전형명 (소분류),수시 모집인원,수시 등록인원,수시 등록률,정시 모집인원,정시 등록인원,정시 등록률,추가 모집인원,추가 등록인원,추가 등록률,최종 모집인원,최종 등록인원,최종 등록률
199,2016,대학교,사립,서울,기존,건국대학교 _본교,정원내,논술위주,일반전형,일반학생,KU논술우수자전형,484,484,100.0,0,0,0.0,0,0,0.0,484,480,99.2
200,2016,대학교,사립,서울,기존,건국대학교 _본교,정원내,학생부위주(종합),고른기회 대상자(정원내),특성화고 등을 졸업한 재직자,KU고른기회전형[유형5-특성화고졸재직자],3,3,100.0,0,0,0.0,0,0,0.0,3,3,100
201,2016,대학교,사립,서울,기존,건국대학교 _본교,정원내,학생부위주(종합),고른기회 대상자(정원내),국가보훈대상자,KU고른기회전형[유형1-사회적배려대상자],48,44,91.7,0,0,0.0,0,0,0.0,48,44,91.7
202,2016,대학교,사립,서울,기존,건국대학교 _본교,정원내,학생부위주(종합),고른기회 대상자(정원내),지역고교 학생(수도권 제외),KU고른기회전형[유형7-지역인재],133,117,88.0,0,0,0.0,0,0,0.0,133,115,86.5
203,2016,대학교,사립,서울,기존,건국대학교 _본교,정원내,실기위주,특기자,체육,KU예체능우수자전형[유형2-체육],15,15,100.0,0,0,0.0,0,0,0.0,15,15,100
204,2016,대학교,사립,서울,기존,건국대학교 _본교,정원내,실기위주,특기자,기타,KU예체능우수자전형[유형1-연기],15,15,100.0,0,0,0.0,0,0,0.0,15,15,100
205,2016,대학교,사립,서울,기존,건국대학교 _본교,정원내,학생부위주(교과),대학별독자적기준,기타,KU교과우수자전형,186,124,66.7,0,0,0.0,0,0,0.0,186,119,64
206,2016,대학교,사립,서울,기존,건국대학교 _본교,정원내,학생부위주(종합),대학별독자적기준,기타,KU자기추천전형,608,575,94.6,0,0,0.0,0,0,0.0,608,573,94.2
207,2016,대학교,사립,서울,기존,건국대학교 _본교,정원내,수능위주,일반전형,일반학생,KU일반전형-다군,0,0,0.0,120,118,98.3,0,0,0.0,108,114,105.6
208,2016,대학교,사립,서울,기존,건국대학교 _본교,정원내,실기위주,일반전형,일반학생,KU일반전형-나군 예술디자인대학 1:4:5 그룹,0,0,0.0,150,150,100.0,0,0,0.0,150,150,100


In [19]:
df['2016'].loc[idx_2016 & (df['2016']['전형명 (대분류)'].str.contains('단원고')
                           | df['2016']['전형명 (중분류)'].str.contains('단원고')
                           | df['2016']['전형명 (소분류)'].str.contains('단원고')), ['최종 등록인원']].sum()

최종 등록인원    9
dtype: int64

In [20]:
df['2016'].loc[~idx_2016 & (df['2016']['전형명 (대분류)'].str.contains('단원고')
                           | df['2016']['전형명 (중분류)'].str.contains('단원고')
                           | df['2016']['전형명 (소분류)'].str.contains('단원고')), ['최종 등록인원']].sum()

최종 등록인원    53
dtype: int64

In [21]:
df['2017'].loc[idx_2017]

,기준년도,학교종류,설립구분,지역,상태,학교,정원구분,전형유형,전형명 (대분류),전형명 (중분류),전형명 (소분류),수시 모집인원,수시 등록인원,수시 등록률,정시 모집인원,정시 등록인원,정시 등록률,추가 모집인원,추가 등록인원,추가 등록률,최종 모집인원,최종 등록인원,최종 등록률
211,2017,대학교,사립,서울,기존,건국대학교 _본교,정원내,논술위주,일반전형,일반학생,KU논술우수자전형,462,459,99.4,0,0,0.0,0,0,0.0,462,456,98.7
212,2017,대학교,사립,서울,기존,건국대학교 _본교,정원내,학생부위주(종합),고른기회 대상자(정원내),특성화고 등을 졸업한 재직자,KU고른기회전형[유형5-특성화고졸재직자],3,3,100.0,0,0,0.0,0,0,0.0,3,3,100
213,2017,대학교,사립,서울,기존,건국대학교 _본교,정원내,학생부위주(종합),고른기회 대상자(정원내),국가보훈대상자,KU고른기회전형[유형1-사회적배려대상자],48,43,89.6,0,0,0.0,0,0,0.0,48,43,89.6
214,2017,대학교,사립,서울,기존,건국대학교 _본교,정원내,실기위주,특기자,미술,KU예체능우수자전형[유형3-디자인],20,20,100.0,0,0,0.0,0,0,0.0,20,20,100
215,2017,대학교,사립,서울,기존,건국대학교 _본교,정원내,실기위주,특기자,체육,KU예체능우수자전형[유형2-체육],13,13,100.0,0,0,0.0,0,0,0.0,13,13,100
216,2017,대학교,사립,서울,기존,건국대학교 _본교,정원내,실기위주,특기자,기타,KU예체능우수자전형[유형1-연기],15,15,100.0,0,0,0.0,0,0,0.0,15,15,100
217,2017,대학교,사립,서울,기존,건국대학교 _본교,정원내,학생부위주(종합),대학별독자적기준,기타,KU학교추천전형,380,301,79.2,0,0,0.0,0,0,0.0,380,299,78.7
218,2017,대학교,사립,서울,기존,건국대학교 _본교,정원내,학생부위주(종합),대학별독자적기준,기타,KU자기추천전형,640,586,91.6,0,0,0.0,0,0,0.0,640,580,90.6
219,2017,대학교,사립,서울,기존,건국대학교 _본교,정원내,실기위주,일반전형,일반학생,KU일반전형-나군(예체능계) 1-4-5 그룹,0,0,0.0,147,147,100.0,0,0,0.0,147,147,100
220,2017,대학교,사립,서울,기존,건국대학교 _본교,정원내,실기위주,일반전형,일반학생,KU일반전형-다군 1-4-5 그룹 연출제작,0,0,0.0,15,15,100.0,0,0,0.0,15,15,100


In [22]:
idx_lack = df['2018']['최종 모집인원'] > df['2018']['최종 등록인원']
df['2018'].loc[idx_2018 & idx_lack]

,기준년도,학교종류,설립구분,지역,상태,학교,정원구분,전형유형,전형명 (대분류),전형명 (중분류),전형명 (소분류),수시 모집인원,수시 등록인원,수시 등록률,정시 모집인원,정시 등록인원,정시 등록률,추가 모집인원,추가 등록인원,추가 등록률,최종 모집인원,최종 등록인원,최종 등록률
211,2018,대학교,사립,서울,기존,건국대학교 _본교,정원내,논술위주,일반전형,일반학생,KU논술우수자전형,465,464,99.8,0,0,0.0,0,0,0.0,465,463,99.6
214,2018,대학교,사립,서울,기존,건국대학교 _본교,정원내,수능위주,일반전형,일반학생,KU일반전형-다군,0,0,0.0,154,144,93.5,0,0,0.0,144,143,99.3
224,2018,대학교,사립,서울,기존,건국대학교 _본교,정원내,실기위주,특기자,체육,KU예체능우수자전형[유형2-체육],13,12,92.3,0,0,0.0,0,0,0.0,13,12,92.3
225,2018,대학교,사립,서울,기존,건국대학교 _본교,정원내,학생부위주(종합),고른기회 대상자(정원내),국가보훈대상자,고른기회전형Ⅰ (국가보훈대상자),20,17,85.0,0,0,0.0,0,0,0.0,20,17,85
227,2018,대학교,사립,서울,기존,건국대학교 _본교,정원내,학생부위주(종합),대학별독자적기준,기타,KU자기추천전형,715,679,95.0,0,0,0.0,0,0,0.0,715,679,95
228,2018,대학교,사립,서울,기존,건국대학교 _본교,정원내,학생부위주(종합),대학별독자적기준,기타,KU학교추천전형,416,367,88.2,0,0,0.0,0,0,0.0,416,366,88
229,2018,대학교,사립,서울,기존,건국대학교 _본교,정원내,학생부위주(종합),대학별독자적기준,기타,고른기회전형Ⅱ,40,36,90.0,0,0,0.0,0,0,0.0,40,35,87.5
233,2018,대학교,사립,서울,기존,건국대학교 _본교,정원외,기타,재외국민 외국인(정원외),재외국민과 외국인(2% 이내),재외국민과 외국인(2% 이내),60,44,73.3,0,0,0.0,0,0,0.0,60,44,73.3
237,2018,대학교,사립,서울,기존,건국대학교 _본교,정원외,학생부위주(종합),기초생활수급자 및 차상위계층(정원외),저소득층(기회균형선발),고른기회전형Ⅰ (기초생활수급자및차상위급여수급자),60,57,95.0,0,0,0.0,0,0,0.0,60,57,95
238,2018,대학교,사립,서울,기존,건국대학교 _본교,정원외,학생부위주(종합),농어촌 학생(정원외),농어촌 학생,고른기회전형Ⅰ (농어촌학생),78,76,97.4,0,0,0.0,0,0,0.0,78,76,97.4


In [23]:
df['2016'].columns

Index(['기준년도', '학교종류', '설립구분', '지역', '상태', '학교', '정원구분', '전형유형', '전형명 (대분류)',
       '전형명 (중분류)', '전형명 (소분류)', '수시 모집인원', '수시 등록인원', '수시 등록률', '정시 모집인원',
       '정시 등록인원', '정시 등록률', '추가 모집인원', '추가 등록인원', '추가 등록률', '최종 모집인원',
       '최종 등록인원', '최종 등록률'],
      dtype='object')

In [24]:
df['2016'].loc[idx_2016].groupby('학교').agg({'학교':'count'})

,학교
학교,
건국대학교 _본교,28
경희대학교 _본교,34
고려대학교 _본교,22
동국대학교 _본교,26
서강대학교 _본교,15
서울대학교 _본교,11
서울시립대학교 _본교,12
성균관대학교 _본교,21
연세대학교 _본교,26
